# Normalizaiton

归一化的目的是:把数据经过某种映射之后，取值限定到一定的范围之内, 比如$[0, 1]$或者$[-1, 1]$。[softmax](../math/softmax.ipynb)它就是常用的一种归一化方法。

那么为什么要进行归一化呢？

样本数据中有许多的特征，我们在训练模型的时候需要将他们转换成数字，然后输入到模型中，但是，通常特征之间的数量级存在差异，那么数量级大的特征就会掩盖住其他的特征。归一化是为了减小特征之间的差异性。

## $\text{Min}$-$\text{Min}$ Normaliztion
其公式为:
$$
x_{new} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}\tag{1}
$$
这种方法将所有的数据归一到$[0, 1]$之间，

## Z-score Normalization
假设样本数据服从正态分布，那么对其归一化可以使用如下的公式
$$
x_{new} = \frac{x - \mu}{\sigma}\tag{2}
$$
Z-score给出了一组数据中各个数据的相对位置。这种归一化方法知识对原分布的线性变换，并不改变其分布的形状

## Batch Normalization

对于一个DNN，零第$l$层的净输入为$z^{(l)}$，神经元的输出为$a^{(l)}$，即
$$
a^{(l)} = f(z^{(l)} = f(Wa^{(l - 1)} + b)\tag{3}
$$
为了提高优化效率，就要使得净输入的分不一致，比如都归一化到正态分布。

假设给定一个包含$K$个样本的小批量集合, 其具体操作如下

$$
\mu = \frac1K \displaystyle\sum_{k = 1}^Kz^{(k, l)}\tag{4}
$$

$$
\sigma= \frac1K \displaystyle\sum_{k = 1}^K(z^{(k, l)} - \mu) \odot (z^{(k, l)} - \mu)\tag{5}
$$
则优化的更新公式如下:

$$
\hat{z}^{(l)} =\alpha\odot \frac{z^{(l)} - \mu}{\sqrt{\sigma^2 + \varepsilon}} + \beta\tag{6}
$$

其中$\alpha$和$\beta$分别代表数据缩放和平移的参数**向量**。

BN是对一个中间层的单个神经元进行归一化操作，因此要求小批量样本的数量不能太小，否则难以计算单个神经元的统计信息. 此外，如果一个神经元的净输入的分布在神经网络中是动态变化的，比如循环神经网络，那么就 无法应用批量归一化操作.

## Layer Normaliztion

层归一化(Layer Normalization)是和批量归一化非常类似的方法. 和批量归一化不同的是，层归一化是对一个中间层的所有神经元进行归一化.

对于一个深度神经网络，令第$l$层神经元的净输入为$z^{(l)}$， 那么有
$$
\mu^{(l)}  = \frac1M_l\displaystyle\sum_{i = 1}^{M_l}z^{(l)}_i\tag{7}
$$


$$
\sigma^{(l)}  = \frac1M_l\displaystyle\sum_{i = 1}^{M_l}(z^{(l)}_i - \mu^{(l)})^2\tag{8}
$$
则更新优化的公式为
$$
\hat{z}^{(l)} = \alpha\odot\frac{z^{(l)} - \mu^{(l)}}{\sqrt{{\sigma^{(l)}}^2 + \varepsilon}} + \beta\tag{9}
$$

如果去看[Transformer](transformer.ipynb)的归一化就会发现，它的公式同上述三个公式一样。

LN和BN的区别是，LN的更新可以发生在一个样本上，而BN的更新必须是一堆样本才行。

## Weight Normalization

权重归一化是对神经网络的连接权重进行归一化，通过再参数化(reparameterization)方法，将连接权重分解为长度和方向两种参数. 根据公式(3)，我们将$W$再参数化为

$$
W_{i, :} = \frac{g_i}{\|v_i \|}v_i, \ \ \ 1 \lt i \lt M_l\tag{10}
$$


其中$W_{i,:}$表示权重$W$的第$i$行(这种索引方式在numpy中使用),新引入的参数$g_i$为标量，$v_i$和$a^{(l - 1)}$维数相同。